This is a test for the weno reconstruction for non-periodic case (which can be used for the periodic case by appropriately populating the ghost zones).

It works OK, BUT, it is only third order accurate for the derivatives. No matter what. To get something better it seems one has to increase the stencils. 

In [ ]:
using Plots
using Revise
# include project utilities
includet("choques_utils.jl")
# complementary_utils.jl is included so its TVD routines are available if you want to call them.

includet("complementary_utils.jl")


In [ ]:
function average_poly_values(p,dx,n,x0)
    # given x^p conpute the average values over the cells
    #[((x0 + dx*n+dx/2)^(p+1) - (x0 + dx*n - dx/2)^(p+1)) / (dx*(p+1)) for n in 0:(n-1)]
    [-((x0 + dx*n)^(p+1) - (x0 + dx*(n+1))^(p+1)) / (dx*(p+1)) for n in 0:(n-1)]
end

In [ ]:
n = 11
L = 4
dx = L / (n-1)
x0 = -L/2
x_centers = ((0:n-2) .* dx) .+ dx/2 .+ x0
x_p = (0:n-1) .* dx .+ x0
p = 4
u_bar_int = average_poly_values(p,dx,n-1, x0)
scatter(x_centers,u_bar_int, label="averages", title = "power of order p=$p" )
scatter!(x_centers, (x_centers).^p, label="point values at centers", color=:red, markersize=2)
plot!(x_centers, (x_centers).^p, label="values at centers", linestyle=:dash)
scatter!(x_p, (x_p).^p, label="point values at edges", color=:green)


In [ ]:
m = n + 4
x0_ext = x0 -3dx
#x0 = -5dx/2

x_centers_ext = ((0:m) .* dx) .+ (x0_ext) .+ dx/2
scatter(x_centers_ext[m - 5:m-2], title="Checking extended centers vs original centers", label="extended centers")
scatter!(x_centers[n-4:n-1], label="original centers")


In [ ]:
u0 = average_poly_values(p,dx,m+1, x0_ext)
@show size(u0)  # Should be m = n +1 + 4 
@show size(x_centers_ext)  # Should be m = n + 5


In [ ]:

scatter(x_centers_ext,u0, title = "Extended function", label="averages over extended domain")
scatter!(x_centers_ext, (x_centers_ext).^p, label="point values at centers", markersize=2)
scatter!(x_p, (x_p).^p, label="point values at edges", color=:green)

In [ ]:
uL, uR, duL, duR = WENOZ_FV_reconstruct_from_averages(u0, dx) # Just to test that it runs

In [ ]:
#length(uL)  # Should be n = 201

In [ ]:
scatter(x_p,uL, label="uL", title = "p = $p", markersize=2)
scatter!(x_p,x_p.^p, label="exact", markersize=2)
scatter!(x_p,uR, label = "uR", markersize=3)

In [ ]:
plot(x_p,uL - x_p.^p, label = "uL error", title = "Reconstruction error for p = $p")
plot!(x_p,uR - x_p.^p, label = "uR error")

In [ ]:
plot(x_p, duL, label="duL")
plot!(x_p, p*x_p.^(p-1), label="exact derivative", linestyle=:dash)
plot!(x_p, duR, label="duR")  

In [ ]:
plot(x_p, duL - p*x_p.^(p-1), label="error duL")
plot!(x_p, duR - p*x_p.^(p-1), label="error duR")  

In [ ]:
@show p, n
@show sum(abs.(uL - x_p.^p)) / n
@show sum(abs.(uR - x_p.^p)) / n
@show sum(abs.(duL - p*x_p.^(p-1))) / n
@show sum(abs.(duR - p*x_p.^(p-1))) / n

(p, n) = (2, 201)
sum(abs.(uL - x .^ p)) / n = 2.5510718312309582e-15
sum(abs.(uR - x .^ p)) / n = 2.577850779418629e-15
sum(abs.(duL - p * x .^ (p - 1))) / n = 0.019999999999993245
sum(abs.(duR - p * x .^ (p - 1))) / n = 0.007999999999994956

(p, n) = (3, 201)
sum(abs.(uL - x .^ p)) / n = 3.504499251805946e-15
sum(abs.(uR - x .^ p)) / n = 3.3198136945253183e-15
sum(abs.(duL - p * x .^ (p - 1))) / n = 0.06030009950248602
sum(abs.(duR - p * x .^ (p - 1))) / n = 0.024124776119409063

(p, n) = (4, 201)
sum(abs.(uL - x .^ p)) / n = 5.829543278212534e-15
sum(abs.(uR - x .^ p)) / n = 5.83344372386125e-15
sum(abs.(duL - p * x .^ (p - 1))) / n = 0.16161439999998206
sum(abs.(duR - p * x .^ (p - 1))) / n = 0.06462213731343115

(p, n) = (5, 201)
sum(abs.(uL - x .^ p)) / n = 6.3999988096115706e-9
sum(abs.(uR - x .^ p)) / n = 6.400001051301997e-9
sum(abs.(duL - p * x .^ (p - 1))) / n = 0.40608230978438853
sum(abs.(duR - p * x .^ (p - 1))) / n = 0.16230796417911367

### Discontinuos Test:

We add two terms one which is discontinuous and one that has discontinuous derivative. We need to compute correctly the averages at those points. 

We aling the cells so that, for x0 = -L/2, one central point is at x = dx/2 so the cell interfases are at x=0 and x = dx.
There for the step function we get that there the the average corresponding to that point is: (1-s)/2
where the discontinuity is at x_i + s*dx/2 so a bit to the right of the middle, if s > 0.

For the $|x-x_2|$ function the average in the cells is: 

$$
\bar(|x - x_2|) = |x - x_2| \;\; \text{for} \;\;\; x - x_2 < dx/2
$$

In the cell where the function becomes zero the average value is:

$$
dx/4*(s^2 + 1)
$$






In [ ]:
#Checking the arrays so that the cell centers are not aligned with the points where we put the discontinuities:

scatter(x_centers_ext[m - 5:m-2])
scatter!(x_centers[n-4:n-1])

#minimum(abs.(x_centers_ext))

In [ ]:
function θ_ave(x,x1,s,dx) 
    if x < x1 - dx
        return 0.0
    elseif x < x1 && x < x1 + dx/2
        return s*(s-1)/2
    else
        return 1.0
    end 
end

θ(x,x1) = ifelse(x < x1, 0.0, 1.0)

Iθ(x,x1) = abs(x - x1)

function Iθ_ave(x,x1,s,dx)
    if abs(x - x1) <= dx/2 
        return (s^2 + 1)*dx/4
    else
        return abs(x - x1)
    end 
end

s1 = 1/2 # we put the discontinuity at x = dx/2 + dx/4 If we change sign we find the larger error at the left reconstruction.
s2 = 1/2 # we put the discontinuity at x = 1 + dx/2 + dx/4
x1 = s1*dx/2
x2 = 1 + dx/2 + s2*dx/2
u_bar = u0 + [θ(xi,x1) + Iθ(xi,x2) for xi in x_centers_ext];
size(u_bar)  # Should be m = n + 5

In [ ]:
plot(x_centers_ext, u_bar, label="initial u_ave")

In [ ]:
uL, uR, duL, duR = WENOZ_FV_reconstruct_from_averages(u_bar, dx) # Just to test that it runs

In [ ]:
plot(x_p, uL, label="uL")
plot!(x_p, uR, label="uR")
u_exact = [xi^p + θ(xi,x1) + abs(xi - x2) for xi in x_p]
plot!(x_p, u_exact, label="exact", linestyle=:dash)

In [ ]:
plot(x_p, uL - u_exact, label="error uL")
plot!(x_p, uR - u_exact, label="error uR")

In [ ]:
du_exact = [p*xi^(p-1) + θ(xi,x2) - θ(x2,xi) for xi in x_p]
plot(x_p, duL, label="duL")
plot!(x_p, duR, label="duR")
plot!(x_p, du_exact, label="exact derivative (without the delta)", linestyle=:dash, legend=:topright)


## New derivatives

In [ ]:
duLnew, duRnew = WENOZ_derivatives_form_uLR(uL, uR, dx)

In [ ]:
#du_exact = [p*xi^(p-1) + 0.0 for xi in x]
plot(x_p, duLnew, label="duL")
plot!(x_p, duRnew, label="duR")
plot!(x_p, du_exact, label="exact derivative (without the delta)", linestyle=:dash, legend=:topright)